In [1]:
%run TAQplots.ipynb

In [3]:
class TAQ2TAQplots:
    def __init__(self, taq_df,
              row_id_col, # unique row id
              sec_col, # column with security code
              link_col, # orderno or some mark linking trades and quotes
              trdno_col
             ):
        self.taq_df = taq_df.copy()
        self.row_id_col = row_id_col
        self.sec_col = sec_col
        self.link_col = link_col
        self.trdno_col = trdno_col
        self.secs = pd.unique(taq_df[sec_col])
        self.vertex_count_dic = {k:v for (k, v) in zip(self.secs, [0 for i in range(len(self.secs))])}
        self.rowid2gkey_dic = {k:v for (k, v) in zip(self.secs, [{} for i in range(len(self.secs))])}
        self.lnk2gkey_dic = {k:v for (k, v) in zip(self.secs, [{} for i in range(len(self.secs))])}
        self.trdno2gkey_dic = {k:v for (k, v) in zip(self.secs, [{} for i in range(len(self.secs))])}
        self.edge_tpls = {k:v for (k, v) in zip(self.secs, [[] for i in range(len(self.secs))])}
        self.rows_to_edges()
        self.Gs = {k:v for (k, v) in zip(self.secs, [Graph(self.edge_tpls[sec],
                                                           self.vertex_count_dic[sec]) for sec in self.secs])}
        self.TAQpps = {k:v for (k, v) in zip(self.secs, [TAQplots(self.Gs[sec]) for sec in self.secs])}

    def sec_rowid_lnk2edges(self, rowid, sec, lnk, trdno): 
        # give 0, 1, 2, ... vertex names to all raw ids and "links" (eg ordernos)
        if rowid not in self.rowid2gkey_dic[sec]:
            self.rowid2gkey_dic[sec][rowid] = self.vertex_count_dic[sec]
            self.vertex_count_dic[sec] += 1
        if lnk not in self.lnk2gkey_dic[sec]:
            self.lnk2gkey_dic[sec][lnk] = self.vertex_count_dic[sec]
            self.vertex_count_dic[sec] += 1
        if trdno not in self.trdno2gkey_dic[sec]:
            self.trdno2gkey_dic[sec][trdno] = self.vertex_count_dic[sec]
            self.vertex_count_dic[sec] += 1
        self.edge_tpls[sec].append((self.rowid2gkey_dic[sec][rowid], 
                               self.lnk2gkey_dic[sec][lnk]))
        if pd.notna(trdno):
            self.edge_tpls[sec].append((self.rowid2gkey_dic[sec][rowid], 
                               self.trdno2gkey_dic[sec][trdno]))

    def rows_to_edges(self):
        [self.sec_rowid_lnk2edges(rowid, sec, lnk, trdno) 
         for rowid, sec, lnk, trdno in zip(self.taq_df[self.row_id_col],
                                    self.taq_df[self.sec_col],
                                    self.taq_df[self.link_col],
                                    self.taq_df[self.trdno_col])]
        
    def TAQplots_df(self):
        taq_df['TAQPLOTNO'] = [t2tpp.TAQpps[sec].TAQplotnos[t2tpp.rowid2gkey_dic[sec][rowid]] 
                                    for (sec, rowid) in zip(OrderLog_df['SECCODE'], 
                                                            OrderLog_df['NO'])]

#OrderLog = 'OrderLog20020812'
OrderLog = 'OrderLog20071001'
OrderLog_df = pd.read_csv('./resources/' + OrderLog + '.txt')

OrderLog_df = OrderLog_df.astype({"NO":"int",
                                  "SECCODE":"string",
                                  "BUYSELL":"string",
                                  "TIME":"string",
                                  "ORDERNO":"int",
                                  "ACTION":"int",
                                  "PRICE":"float",
                                  "VOLUME":"int",
                                  "TRADENO":pd.Int64Dtype(),
                                  "TRADEPRICE":"float"
                                 })
OrderLog_df.head()

In [ ]:
t2tpp = TAQ2TAQplots(OrderLog_df, 'NO', 'SECCODE', 'ORDERNO', 'TRADENO')
OrderLog_df['TAQPLOTNO'] = [t2tpp.TAQpps[sec].TAQplotnos[t2tpp.rowid2gkey_dic[sec][rowid]] 
                                    for (sec, rowid) in zip(OrderLog_df['SECCODE'], 
                                                            OrderLog_df['NO'])]
OrderLog_df.to_csv('./resources/' + OrderLog + '.csv')
OrderLog_df

plotno_30_df = OrderLog_df.loc[(OrderLog_df.SECCODE == 'LKOH') & (OrderLog_df.TAQPLOTNO == 1599)]
plotno_30_df